In [ ]:
import cv2
import numpy as np
import pickle
import os
import random
from tensorflow.keras.utils import to_categorical
import shutil

In [ ]:
shutil.unpack_archive("/content/train_test.zip", "/content/")

In [ ]:
DATADIR = '/content/train_test'
CATEGORIES = ['CU','MC','ML','L']
training_data = []

In [ ]:
for category in CATEGORIES:
  path = os.path.join(DATADIR,category)
  class_num=CATEGORIES.index(category)
  for img in os.listdir(path):
    try:
      img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
      #img_array = img_array.ravel()
      training_data.append([img_array,class_num])

    except Exception as e:
      print('error')
      pass

In [ ]:
len(training_data)

872

In [ ]:
random.shuffle(training_data)

X = []
y = []

In [ ]:
for features, label in training_data:
  X.append(features)
  y.append(label)

In [ ]:
y=to_categorical(y)

In [ ]:
for i in X[:10]:
  print(i.shape)

(180, 320)
(180, 320)
(180, 320)
(180, 320)
(180, 320)
(180, 320)
(180, 320)
(180, 320)
(180, 320)
(180, 320)


In [ ]:
X = np.array(X).reshape(-1,320,180,1)
y = np.array(y)

In [ ]:
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pickle
import numpy as np

In [ ]:
X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train/255.0
X_test = X_test/255.0

In [ ]:
X_train.shape[1:]

(320, 180, 1)

In [ ]:
model = keras.Sequential([
  keras.layers.Conv2D(128,(3,3),activation='relu',input_shape=X_train.shape[1:]),
  keras.layers.MaxPool2D(2,2),
  keras.layers.Conv2D(64,(3,3),activation='relu'),
  keras.layers.MaxPool2D(2,2),

  keras.layers.Flatten(),
  keras.layers.Dense(64,activation='relu'),
  keras.layers.Dense(32,activation='relu'),
  #keras.layers.Dropout(0.4),
  keras.layers.Dense(4,activation='softmax')
])

In [ ]:
from keras.optimizers import SGD
#opt = SGD(lr=0.1)
model.compile(loss="categorical_crossentropy", optimizer='sgd', metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train, batch_size=3, epochs=25, validation_data=(X_test, y_test))

Epoch 1/25
233/233 [==============================] - 4s 17ms/step - loss: 1.3837 - accuracy: 0.2569 - val_loss: 1.3625 - val_accuracy: 0.3714
Epoch 2/25
233/233 [==============================] - 4s 17ms/step - loss: 1.3589 - accuracy: 0.3214 - val_loss: 1.3469 - val_accuracy: 0.3943
Epoch 3/25
233/233 [==============================] - 4s 17ms/step - loss: 1.3265 - accuracy: 0.3753 - val_loss: 1.3653 - val_accuracy: 0.3200
Epoch 4/25
233/233 [==============================] - 4s 16ms/step - loss: 1.3296 - accuracy: 0.3489 - val_loss: 1.3798 - val_accuracy: 0.3943
Epoch 5/25
233/233 [==============================] - 4s 16ms/step - loss: 1.2965 - accuracy: 0.4012 - val_loss: 1.3118 - val_accuracy: 0.4114
Epoch 6/25
233/233 [==============================] - 4s 17ms/step - loss: 1.2445 - accuracy: 0.4238 - val_loss: 1.4235 - val_accuracy: 0.2743
Epoch 7/25
233/233 [==============================] - 4s 16ms/step - loss: 1.2605 - accuracy: 0.4407 - val_loss: 1.3091 - val_accuracy: 0.4286

In [ ]:
model.evaluate(X,y)

28/28 [==============================] - 1s 46ms/step - loss: 227.7557 - accuracy: 0.7144


[227.75567626953125, 0.7144495248794556]

In [ ]:
model.save("Shot Class model.h5")

In [ ]:
from keras.models import load_model
types = load_model("Shot Class model.h5")

In [ ]:
files[0:4]

['The.Grand.Budapest.Hotel.2014.DVDRip.XviD-EVO-Scene-0040-01.jpg',
 'The.Grand.Budapest.Hotel.2014.DVDRip.XviD-EVO-Scene-0312-02.jpg',
 'Suspiria.1977.HUN.BDRiP.x264-ARROW-Scene-090-01.jpg',
 'The.Grand.Budapest.Hotel.2014.DVDRip.XviD-EVO-Scene-0336-02.jpg']

In [ ]:
files = os.listdir("/content/test_type")
CATEGORIES= ["CU","MCU","ML","Long"]
#prediction = angle.predict(files[0])
#print(prediction)
#return position of max
for file in files[6:]:
  os.chdir(r"/content/test_t")
  img = cv2.imread(file,cv2.IMREAD_GRAYSCALE)
  #print(img)
  img = cv2.resize(img,(320,180))
  new_img = img.reshape(-1,320,180,1)
  prediction = types.predict_classes(new_img)
  print(file)
  print(prediction)
  print(CATEGORIES[int(prediction)]) 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Suspiria.1977.HUN.BDRiP.x264-ARROW-Scene-068-01.jpg
[3]
Long
The.Grand.Budapest.Hotel.2014.DVDRip.XviD-EVO-Scene-0029-01.jpg
[3]
Long
Phantom.Thread.2017.BDRip.x264-GECKOS-Scene-384-01.jpg
[2]
ML
The.Grand.Budapest.Hotel.2014.DVDRip.XviD-EVO-Scene-0011-01.jpg
[3]
Long
Suspiria.1977.HUN.BDRiP.x264-ARROW-Scene-259-01.jpg
[1]
MCU
The.Grand.Budapest.Hotel.2014.DVDRip.XviD-EVO-Scene-0310-02.jpg
[1]
MCU
Phantom.Thread.2017.BDRip.x264-GECKOS-Scene-268-01.jpg
[2]
ML
Suspiria.1977.HUN.BDRiP.x264-ARROW-Scene-086-01.jpg
[2]
ML
Phantom.Thread.2017.BDRip.x264-GECKOS-Scene-266-01.jpg
[3]
Long
Phantom.Thread.2017.BDRip.x264-GECKOS-Scene-078-01.jpg
[3]
Long
Phantom.Thread.2017.BDRip.x264-GECKOS-Scene-028-01.jpg
[2]
ML
Phantom.Thread.2017.BDRip.x264-GECKOS-Scene-032-01.jpg
[2]
ML
Phantom.Thread.2017.BDRip.x264-GECKOS-Scene-065-01.jpg
[1]
MCU
Phantom.Thread.2017.BDRip.x264-GECKOS-Scene-365-01.jpg
[3]
Long
Suspiria.1977.HUN.BDRiP.x264-ARROW-Scene-279-01.jpg
[3]
Long


Trying a different method

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import KFold
import numpy as np

# Model configuration
batch_size = 3
img_width, img_height, img_num_channels = 320, 180, 1
loss_function = categorical_crossentropy
no_classes = 4
no_epochs = 25
optimizer = SGD()
verbosity = 1
num_folds = 5



# Determine shape of the data
input_shape = (img_width, img_height, img_num_channels)

# Parse numbers as floats
#input_train = input_train.astype('float32')
#input_test = input_test.astype('float32')

# Normalize data
input_train = X_train / 255
input_test = X_test / 255
target_train = y_train
target_test = y_test

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

# Merge inputs and targets
inputs = np.concatenate((input_train, input_test), axis=0)
targets = np.concatenate((target_train, target_test), axis=0)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
  model = Sequential()
  model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(4, activation='softmax'))



  # Compile the model
  model.compile(loss='categorical_crossentropy',
                optimizer='sgd',
                metrics=['accuracy'])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=no_epochs,
              #verbose=verbosity,
              validation_data=(X_test, y_test))

  # Generate generalization metrics
  scores = model.evaluate(X_test, y_test, verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/25
233/233 [==============================] - 5s 17ms/step - loss: 1.3824 - accuracy: 0.2940 - val_loss: 1.3930 - val_accuracy: 0.2400
Epoch 2/25
233/233 [==============================] - 4s 16ms/step - loss: 1.3363 - accuracy: 0.3867 - val_loss: 1.3469 - val_accuracy: 0.3486
Epoch 3/25
233/233 [==============================] - 4s 17ms/step - loss: 1.3174 - accuracy: 0.3789 - val_loss: 1.3440 - val_accuracy: 0.3943
Epoch 4/25
233/233 [==============================] - 4s 17ms/step - loss: 1.3318 - accuracy: 0.3478 - val_loss: 1.3180 - val_accuracy: 0.3543
Epoch 5/25
233/233 [==============================] - 4s 16ms/step - loss: 1.2844 - accuracy: 0.3801 - val_loss: 1.3271 - val_accuracy: 0.3886
Epoch 6/25
233/233 [==============================] - 4s 17ms/step - loss: 1.2687 - accuracy: 0.4159 - val_loss: 1.3112 - val_accuracy: 0.3486
Epoch 7/25
233/233 [=========================

In [ ]:
model.evaluate(X,y)

28/28 [==============================] - 1s 45ms/step - loss: 268.2079 - accuracy: 0.7672


[268.2079162597656, 0.767201840877533]